### 1. Importation des bibliothèques nécessaires

### 2. Charger le jeu de données

### 3. Prétraitement des données



### 4. Construction de l'Autoencoder

Un autoencoder est un type de réseau de neurones qui apprend à compresser les données dans une représentation latente de taille réduite (appelé "code"), puis à les reconstruire à partir de ce code

### 5. Encoder les données

Après l'entraînement de l'autoencoder, nous utilisons uniquement la partie "encoder" du modèle pour obtenir les caractéristiques réduites.


### 6. Clustering sur les données encodées

Maintenant que les données ont été réduites à un espace de plus faible dimension, nous appliquons un algorithme de clustering comme K-Means.


### 7. Visualisation des clusters (optionnel)

Si tes données ont une faible dimension (par exemple, 2 ou 3 dimensions après encodage), tu peux visualiser les clusters.


### 8. Sauvegarde des résultats

### Résumé

Ce script :
1. Charge et normalise les données.
2. Utilise un autoencoder pour réduire la dimension des données.
3. Applique K-Means sur les données encodées pour identifier les clusters.
4. Visualise et sauvegarde les résultats.



### Améliorer dernier version 

In [25]:
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers, models, optimizers, regularizers, losses
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [30]:


# Construction du modèle autoencodeur
input_layer = layers.Input(shape=(df_pca.shape[1],))
encoded = layers.Dense(16, activation='leaky_relu', activity_regularizer=regularizers.l2(1e-4))(input_layer)
encoded = layers.GaussianNoise(0.1)(encoded)  # Ajout de bruit pour introduire de la variabilité
encoded = layers.Dropout(0.5)(encoded)
encoded = layers.Dense(10, activation='leaky_relu')(encoded)
decoded = layers.Dense(16, activation='leaky_relu')(encoded)
decoded = layers.Dense(df_pca.shape[1], activation='sigmoid')(decoded)

# Modèle autoencodeur
autoencoder = models.Model(input_layer, decoded)

# Cyclical Learning Rate pour ajustement dynamique du taux d'apprentissage
initial_lr = 1e-4
lr_schedule = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate=initial_lr,
    maximal_learning_rate=5e-4,
    scale_fn=lambda x: 1 / (2.0 ** (x - 1)),
    step_size=2000
)

# Compilation du modèle avec MSLE pour capturer les petites erreurs
autoencoder.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule), loss=losses.MeanSquaredLogarithmicError())

# Callbacks pour ajuster l'apprentissage et stopper tôt si nécessaire
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Entraînement du modèle
history = autoencoder.fit(
    df_pca, df_pca,
    epochs=17,  # Utiliser EarlyStopping pour ajuster dynamiquement
    batch_size=32, shuffle=True, validation_split=0.2,
    callbacks=[reduce_lr, early_stopping]
)

# Affichage des courbes de perte pour évaluer l'apprentissage
plt.figure(figsize=(14, 6))
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Courbes de Perte d’Entraînement et de Validation')
plt.show()

# Définir le modèle encodeur à partir de l'autoencodeur
encoder = models.Model(inputs=input_layer, outputs=encoded)

# Encoder les données pour obtenir les représentations latentes
encoded_data = encoder.predict(df_pca)

# Appliquer K-Means sur les représentations encodées pour le clustering
kmeans = KMeans(n_clusters=6, n_init=10)
clusters = kmeans.fit_predict(encoded_data)

# Réduction dimensionnelle avec PCA pour la visualisation
pca_result = PCA(n_components=2).fit_transform(encoded_data)

# Ajouter les résultats au DataFrame pour la visualisation des clusters
df['pca1'], df['pca2'] = pca_result[:, 0], pca_result[:, 1]
df['cluster'] = clusters

# Visualisation des clusters après réduction dimensionnelle
plt.figure(figsize=(10, 8))
sns.scatterplot(x='pca1', y='pca2', hue='cluster', data=df, palette='Set2', legend='full')
plt.title('Visualisation des clusters après réduction dimensionnelle')
plt.show()


Epoch 1/17
2122/2122 [==============================] - 21s 9ms/step - loss: 0.0820 - val_loss: 0.0327 - lr: 4.7580e-04
Epoch 2/17
2115/2122 [============================>.] - ETA: 0s - loss: 0.0400

In [29]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Calculer les métriques de segmentation
silhouette_avg = silhouette_score(encoded_data, clusters)
calinski_harabasz = calinski_harabasz_score(encoded_data, clusters)
davies_bouldin = davies_bouldin_score(encoded_data, clusters)

print(f"Silhouette Score: {silhouette_avg:.2f}")
print(f"Calinski-Harabasz Index: {calinski_harabasz:.2f}")
print(f"Davies-Bouldin Index: {davies_bouldin:.2f}")


Silhouette Score: 0.58
Calinski-Harabasz Index: 20130.72
Davies-Bouldin Index: 1.09


##  Interprétation des Résultats de Segmentation :
Silhouette Score : Plus proche de +1 est idéal, indiquant des clusters bien définis.

Calinski-Harabasz Index : Des valeurs plus élevées suggèrent une meilleure séparation.

Davies-Bouldin Index : Plus faible est meilleur, indiquant des clusters bien séparés